# TP2 : Construir uma classe Python que implemente o DSA

## Função auxiliar

Foi utilizada uma função auxiliar para gerar um número primo dado o seu tamanho, retornando 
o primo com o respetivo tamanho indicado.


In [106]:
def rprime(l):
      return random_prime(2^l-1,True,2^(l-1))


## Definição do problema
### Alinea a)

Os primos **p** e **q** são passados como parâmetros como é pedido e para a sua geração
começamos por gerar **q** aleatoriamente e,posteriormente, calcular para sucessivos valores
de "t" o valor *p = 1 + q << t * até que p seja primo e tenha o tamanho certo.

### Alinea b)

Antes de se proceder à implementação das funçoes para assinar e verificar a assinatura foi 
necessário guardar as variáveis que seriam utilizadas:
    
* **x** : *Private Key* (inteiro) gerada aleatoriamente entre [1,q]
* **y** : *Public Key* gerada a partir da private key (g^x % p) 
* **g** : é o nosso gerador para as chaves
* **(r,sig)** : Assinatura do documento (inteiro,inteiro)
    
A primeira variável é utilizada para assinar a mensagem, sendo o resultado retornado na forma (r,sig).
As variáveis y,g,p e q são utilizadas na verificação da assinatura.

Para o algoritmo de assinar e verificar baseamo-nos num documento encontrado na internet.


In [107]:
from random import randint
from fractions import gcd
from sage.rings.all import Integer


class DSA:
    def __init__(self, pbit=1024, qbit=160):
        q = rprime(qbit)
        t=1
        p = 1+2^t * q
        while not is_prime(p) and p.nbits() < pbit: #enquanto nao é primo nem tem o tamanho pretendido
            t+=1
            p = 1+2^t * q
 
        P = Integers(p) #anel de inteiro modulo p
        Q = Integers(q) #anel de inteiro modulo q
        
        for x in Primes(): #testar a primalidade de x
            a = P(x)
            if a.multiplicative_order() == p-1:
                break
        
        g = pow(a,2**t)#gerador
      
        x = Q.random_element()#chave privada
        y = pow(g,x,p)#chave publica
         
        self.sign_k = x
        self.verify_k = (y,g,p,q)
        
    def sign_key(self):
        return self.sign_k
    
    def verify_key(self):
        return self.verify_k

    def sign(self,key,msg) :
        (_,g,p,q) = self.verify_k
        P = Integers(p)
        Q = Integers(q)
        r=0
        while r==0: #enquanto o r for 0 calcular k novamente
            k = Q.random_element()#nonce
            r = Q(P(g^k)) 
        Z = IntegerRing() #retorna o anel inteiro
        m = Z(msg.encode("hex"), base=16) #hash  
        sig = Q((1/k) *(m + key * r))
        return r,sig

    def verify(self,verkey,signature,msg) :
        (y,g,p,q) = verkey
        P = Integers(p)
        Q = Integers(q)
        r,sig = signature
        w = Q(1/sig)
        Z = IntegerRing()
        m = Z(msg.encode("hex"), base=16) #hash
        u1 = Q(m * w)
        u2 = Q(r*w)
        v = Q(P((g^u1) * (y^u2)))
        if v != Q(r):
            raise ValueError("Assinatura Invalida")


In [108]:
dsa = DSA()
msg = "Esta mensagem vai ser assinada e verificada com sucesso" 
print "Mensagem:", msg

sign_key = dsa.sign_key()
verify_key = dsa.verify_key()

signature = dsa.sign(sign_key,msg)
print "Signature:", signature

msg = dsa.verify(verify_key,signature, msg)
print "Verificação Bem Sucedida"


Mensagem: Esta mensagem vai ser assinada e verificada com sucesso
Signature: (434973058844166455056698842466393492032303506118, 327697310434326153499622947746244301059307580694)
Verificação Bem Sucedida
